In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190915155521-0003
KERNEL_ID = bd8e0208-8beb-465b-8024-429aee40fa2f


# IEEE CIS Fraud Detection - ETL

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pixiedust.display import *

Pixiedust database opened successfully


## Training Set

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# We have already ETLed the training set in data exploration
df_train = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('transaction_id_train.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

## Test Set

In [5]:
test_identity = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test_identity.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [6]:
test_transaction = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test_transaction.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [7]:
test_identity.createOrReplaceTempView("test_identity")
test_transaction.createOrReplaceTempView("test_transaction")

### Joining Transaction and Identity

In [8]:
columns = test_identity.columns[1:]
queryCols = ''
for col in columns:
    queryCols += ', b.'+col
queryCols

', b.id_01, b.id_02, b.id_03, b.id_04, b.id_05, b.id_06, b.id_07, b.id_08, b.id_09, b.id_10, b.id_11, b.id_12, b.id_13, b.id_14, b.id_15, b.id_16, b.id_17, b.id_18, b.id_19, b.id_20, b.id_21, b.id_22, b.id_23, b.id_24, b.id_25, b.id_26, b.id_27, b.id_28, b.id_29, b.id_30, b.id_31, b.id_32, b.id_33, b.id_34, b.id_35, b.id_36, b.id_37, b.id_38, b.DeviceType, b.DeviceInfo'

In [9]:
query = f"""
SELECT 
    a.* {queryCols} 
    FROM test_transaction a
    INNER JOIN test_identity b
    ON a.transactionID = b.transactionID
"""

transaction_identity = sql(query)
transaction_identity.createOrReplaceTempView("transaction_identity")

In [10]:
display(transaction_identity)

TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
3713171,20054054,75.0,R,15592,494.0,150.0,mastercard,102.0,credit,299.0,87.0,None,36.0,yahoo.com,yahoo.com,2.0,1.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,176435.0,None,None,0.0,0.0,None,None,None,None,100.0,Found,52.0,-480.0,New,NotFound,166.0,None,215.0,222.0,None,None,None,None,None,None,None,New,NotFound,Windo

In [11]:
# select only columns in both training and test
col_train = df_train.columns

In [12]:
col_train

['D3',
 'D2',
 'D5',
 'D7',
 'dist2',
 'id_18',
 'D12',
 'D13',
 'D14',
 'V50',
 'V38',
 'V40',
 'V52',
 'V51',
 'V37',
 'V47',
 'V46',
 'V44',
 'V43',
 'V42',
 'V41',
 'V45',
 'V39',
 'D4',
 'id_04',
 'id_03',
 'D6',
 'V56',
 'V55',
 'V57',
 'V64',
 'V58',
 'V71',
 'V60',
 'V61',
 'V62',
 'V63',
 'V72',
 'V59',
 'V68',
 'V65',
 'V74',
 'V73',
 'V67',
 'V66',
 'V92',
 'V85',
 'V93',
 'V94',
 'V88',
 'V87',
 'V86',
 'V78',
 'V84',
 'V83',
 'V82',
 'V80',
 'V79',
 'V77',
 'V81',
 'V89',
 'D15',
 'V34',
 'V22',
 'V33',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V32',
 'V31',
 'D10',
 'D9',
 'D8',
 'id_10',
 'id_09',
 'id_32',
 'id_14',
 'V147',
 'V148',
 'V149',
 'V153',
 'V154',
 'V155',
 'V142',
 'V141',
 'V156',
 'V157',
 'V140',
 'V146',
 'V161',
 'V139',
 'V138',
 'V163',
 'V162',
 'V158',
 'V164',
 'V143',
 'V144',
 'V145',
 'V150',
 'V151',
 'V152',
 'V159',
 'V165',
 'V160',
 'V166',
 'V331',
 'V333',
 'V334',


In [13]:
col_query = col_train[0]
for col in col_train[1:]:
    if col != 'isFraud':
        col_query = col_query + ', ' + col
col_query

'D3, D2, D5, D7, dist2, id_18, D12, D13, D14, V50, V38, V40, V52, V51, V37, V47, V46, V44, V43, V42, V41, V45, V39, D4, id_04, id_03, D6, V56, V55, V57, V64, V58, V71, V60, V61, V62, V63, V72, V59, V68, V65, V74, V73, V67, V66, V92, V85, V93, V94, V88, V87, V86, V78, V84, V83, V82, V80, V79, V77, V81, V89, D15, V34, V22, V33, V14, V15, V16, V17, V18, V19, V20, V21, V23, V24, V25, V26, V27, V28, V32, V31, D10, D9, D8, id_10, id_09, id_32, id_14, V147, V148, V149, V153, V154, V155, V142, V141, V156, V157, V140, V146, V161, V139, V138, V163, V162, V158, V164, V143, V144, V145, V150, V151, V152, V159, V165, V160, V166, V331, V333, V334, V332, V335, V336, V337, V324, V325, V330, V329, V339, V328, V327, V326, V322, V323, V338, addr2, addr1, id_13, V253, V269, V274, V275, V276, V277, V278, V252, V235, V233, V232, V231, V230, V229, V228, V226, V217, V218, V225, V224, V219, V273, V223, V268, V258, V249, V248, V247, V246, V244, V243, V242, V241, V240, V237, V267, V254, V257, V236, V266, V265, V2

In [14]:
query = f"""
SELECT 
    {col_query}
    FROM transaction_identity
"""

transaction_identity2 = sql(query)
transaction_identity2.createOrReplaceTempView("transaction_identity2")

In [15]:
# save dataset
transaction_identity_pd = transaction_identity2.toPandas()

# save file
# Save dataframe as csv file to storage
project.save_data(data=transaction_identity_pd.to_csv(index=False), file_name='transaction_id_test.csv',overwrite=True)

{'file_name': 'transaction_id_test.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'default-donotdelete-pr-l16zufkrkht8vm',
 'asset_id': 'bca9469a-4fe8-4c9a-82ae-18192c64293c'}